In [2]:
import cv2  # OpenCVで動画と画像処理
import torch  # PyTorchでYOLOv5モデル利用
import sys

# ========= 1. YOLOv5モデルのロード =========
#   yolov5s.ptは自動でダウンロードされます
#   deviceを'cpu'に明示することでCPU専用実行になります
model = torch.hub.load(
    './yolov5',          # クローンしたYOLOv5リポジトリのパス
    'custom',            # カスタムモデル（yolov5s等も可）
    path='yolov5s.pt',   # 学習済みモデル（デフォルトでOK）
    source='local',      # ローカルファイルとして読み込む
    device='cpu'         # CPU強制
)

# ラベル（人、車…などの名前リスト）
class_names = model.names

# ========= 2. 動画ファイルの読み込み =========
video_path = 'data/sample.mp4' # 例：'input.mp4'  カメラなら 0
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("動画が開けません。ファイルパスを確認してください。")
    sys.exit(1)

while True:
    ret, frame = cap.read()
    if not ret:
        break  # 動画の最後でループ終了

    # ========= 3. 推論（物体検出） =========
    # OpenCVはBGR⇒YOLOはRGB
    results = model(frame[..., ::-1])  # YOLOv5に画像を渡して検出

    # ========= 4. 結果をフレームに描画 =========
    # 各物体ごとに検出ボックス＆ラベルを追加
    for *box, conf, cls in results.xyxy[0].tolist():
        # 座標とラベル
        x1, y1, x2, y2 = map(int, box)
        label = class_names[int(cls)]
        score = float(conf)
        # 検出枠
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # ラベル
        cv2.putText(
            frame,
            f"{label}: {score:.2f}",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2
        )

    # ========= 5. ウィンドウ表示 =========
    cv2.imshow("YOLOv5 Detection", frame)
    # qキーで途中終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ========= 6. 後処理 =========
cap.release()
cv2.destroyAllWindows()

YOLOv5 🚀 v7.0-425-g85acef3a Python-3.12.3 torch-2.8.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocas